# Demo: A Deeper Dive into Floating Point Types
We work with numerical data in many tasks in science, technology, engineering, and mathematics (STEM). Thus, let's explore common floating-point data types, how they are represented in memory, and the precision associated with each type.
* __Key (surprising) fact__: All floating point numbers are approximations! A floating-point number uses a _fixed number of bits_, i.e., memory to store a value, so it can only represent a finite set of rational values rather than the continuum of real numbers. Thus, any real number must be rounded to the nearest floating-point value, making every floating-point value an approximation.

The typical floating-point number types you will likely encounter in applications are `Float16`, `Float32`, and `Float64`. But what do these numbers mean? For example, what do the `16`, `32`, and `64` mean in `FloatXX`, what precision can these numbers describe, and how are they represented in memory?

To answer these questions, we need to establish a few ground truths. First, on regular, i.e., non-quantum hardware, all floating-point numbers are stored as binary numbers, i.e., numbers written in base `2`. Let's start by looking at the representation of base `b` numbers.

## Base b representation of numbers
A number in base $b$ is represented by a finite sequence of digits $(d_{n}d_{n-1}\dots{d_{1}}d_{0})_{b}$ where each digit $d_{i}$ satifies $0\leq{d_{i}}<b$. The value (in base 10) of a base $b$ number is the positional sum:
$$
\begin{align*}
\underbrace{(d_{n}d_{n-1}\dots{d_{1}}d_{0})_{b}}_{\text{base b}} = \underbrace{\sum_{i=0}^{n}d_{i}b^{i}}_{\text{value in base 10}}
\end{align*}
$$
Let's use integers for a few examples to better understand this expression (and then we'll move on to floating point numbers). 

Consider an `Int64` number. We know that memory storage on modern non-quantum hardware is binary, i.e., base $b = 2$; thus, all the digits $d_{i}$ must be $0\leq{d}_{i}<2$. However, how many digits do we have, i.e., what is the value of $n$? This is the _word size_, i.e., the `64` in `Int64`.
* __Hmmm__. Didn't we already see that? Yes, it's the length of the string output from [the `bitstring(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring)! Let's count the number of zero digits and the number of one digits of a test 64-bit integer using the [`count_zeros(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.count_zeros) and [`count_ones(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.count_ones)

In [56]:
let
    wordsize = 64; # default word size
    x = 24; # Int64 value by default
    n = count_zeros(x) + count_ones(x) # this counts 0's and 1's (doesn't give any info about position)
    @assert wordsize == n # see https://docs.julialang.org/en/v1/base/base/#Base.@assert
end

The positions of the `0` and `1` values in the binary number give us the number's value. Suppose we get the bit pattern, i.e., the positions of the digits, using [the `bitstring(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring) and save this value in the `s::String` variable:

In [91]:
s = let
    x = 257; # Int64 value by default
    s = bitstring(x)
end

"0000000000000000000000000000000000000000000000000000000100000001"

For the binary string $s$, from the right to the left, we add powers of 2, i.e., the $b^{i}$ terms in the sum, for positions whose digit is a `1`. Let's make this more concrete. 
* __Hypothesis__: We should be able to _process_ the string $s$, i.e., compute the positional sum of $s$, and get back the value of the integer that generated it. To do this, we'll need to use a few tricks that we haven't discussed yet. Don't worry about how we do it; let's just see if our hypothesis is true or false.

In [93]:
let

    b = 2; # what base do we have?
    s′ = reverse(s) |> collect .|> x-> parse(Int64,x) # fancy. Nothing to see here, move along (for now anyway).
    count = 0; # if this works, when we are finished, this should be our original number
    for i ∈ eachindex(s′)
        dᵢ = s′[i];
        count+= (dᵢ)*(b^(i-1)) # Hmmm. This looks like the position sum, but why i -1?
    end
    println("Was your number $(count)?")
end

Was your number 257?


## Floating point numbers
Now that we have seen how integers are laid out in memory, let's try to understand the memory layout expressions of floating types `Float16`, `Float32`, and `Float64`. However, before we do that, what is the use case for three floating-point types?

Using three floating-point types lets us balance precision and resource usage for different applications: 
* `Float16` (half-precision) minimizes memory footprint at the expense of precision. This type is ideal for large-scale machine learning inference or graphics where fine precision isn’t critical, and we have many values to store in memory.
* `Float32` (single-precision) offers a good compromise of speed and accuracy for most numerical and real-time workloads.
* `Float64` (double-precision) provides the high precision and wide exponent range needed in scientific computing, simulations, and financial modeling, where rounding errors must be tightly controlled.

Hypothetically, suppose we need more precision than `Float64`, are there bigger values? Yes, specialized types in Julia (and Python) let you go above `Float64`. For example, [the `Quadmath.jl` package](https://github.com/JuliaMath/Quadmath.jl) in Julia implements a `Float128` type if you need precision beyond `Float64`.

### Float16
Fill me in

### Float32
Fill me in

### Float64
A 64-bit floating point number $x\in\mathbb{R}$ is represented in memory as:
$$
\begin{align*}
x = -1^{s}\times\,1.\underbrace{d_{51}d_{50}\dots{d_{0}}}_{52\,\text{fraction bits}}\times{2^{E-1023}}
\end{align*}
$$
where $s\in\{0,1\}$ denotes the sign-bit (bit `63`), $1.d_{51}d_{50}\dots{d_{0}}$ form the significand and  $E$ is the unsigned 11$\times$bit stored exponent (value of bits $52\rightarrow{62}$) given by: $E = \sum_{i=0}^{10}d_{52+i}2^{10-i}$. 

Let's compute the components and see if we get the original number back. Use [the `bitstring(...)` method](https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring) to generate the bits in the number.

In [153]:
d₃, x₃ = let

    d = Dict{Int64,Int64}();
    wordsize = 64; 
    x = 1234.567; # example 64-bit floating point number
    a = bitstring(x) |> reverse |> collect .|> v-> parse(Int64,v) # fancy. Nothing to see here, move along (for now anyway).
    
    # put stuff in the dictionary
    for i ∈ 0:(wordsize-1)
        d[i] = a[i+1];
    end
    d,x
end

(Dict(5 => 0, 56 => 0, 35 => 0, 55 => 1, 60 => 0, 30 => 0, 32 => 0, 6 => 1, 45 => 0, 4 => 1…), 1234.567)

In [155]:
bitstring(x₃)

"0100000010010011010010100100010010011011101001011110001101010100"

In [161]:
d₃[4]

1

Compute the three components, the sign bit $s$, the significand and the exponent $E$:

In [163]:
signvalue, significandvalue, E = let

    # initialize -
    base = 2.0;
    wordsize = 64;
    d = d₃; # bit pattern dictionary

    # compute the sign bit
    signvalue = (-1)^d[wordsize-1];

    # exponent -
    E = 0;
    exponentrange = range(52,stop=62,step=1) |> collect;
    for i ∈ eachindex(exponentrange)
        j = exponentrange[i];
        E += d[j]*(base^(10-i))
    end

    # significand -
    significandvalue = 0;
    significandrange = range(0,stop=51,step=1) |> collect;
    for i ∈ significandrange
        significandvalue+= d[i]*(base^i)
    end

    signvalue, significandvalue, E
end

(1, 9.26083459703636e14, 576.5)

In [165]:
significand(x₃)

1.2056318359375

In [169]:
exponent(x₃)

10